In [53]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from webdriver_manager.firefox import GeckoDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time

In [54]:
import pandas as pd
from datetime import datetime

autentica = pd.read_excel('ls.xlsx')
login = str(autentica['login'][0])
senha = str(autentica['senha'][0])

nome = 'deposito_itens'
indisponivel = 'indisponivel'

data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

In [55]:

def abrir_suap():
    
    global nav
    #nav = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    nav = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#     nav = webdriver.Chrome(ChromeDriverManager().install())
    nav.set_page_load_timeout(60)
    nav.get("https://suap.ifpe.edu.br/accounts/login/?next=/")
    
    # inserindo usuário
    nav.find_element(By.XPATH, '//*[@id="id_username"]').click()
    nav.find_element(By.XPATH, '//*[@id="id_username"]').send_keys(login)

    # inserindo senha 
    nav.find_element(By.XPATH, '//*[@id="id_password"]').click()
    nav.find_element(By.XPATH, '//*[@id="id_password"]').send_keys(senha)

    nav.find_element(By.XPATH, '//*[@id="ok"]').click()

In [56]:
# extraindo para lista
def table_to_list():
    tabela = []

    lista = nav.find_element(By.TAG_NAME, 'table')
    td = lista.find_elements(By.TAG_NAME, 'td')

    for i in td:
        tabela.append(i.text)

    return tabela

In [57]:
def build_df(lista):
    df_tabela = pd.DataFrame()


    tombos = []
    ele_des = []
    descricao = []
    cargas = []
    c_contabil = []
    salas = []
    dt_entrada = []
    fornecedores = []
    valores = []

    #  remove o primeiro elemento ta tabela, informação quebrada    
    for excluir in range(0, len(lista), 1201):
        lista.pop(excluir)

    for tombo in range(1, len(lista)+1, 12):
        if lista[tombo] == '':
            lista.pop(tombo)

    # listas para montar o data frame
    for tombo in range(1, len(lista), 12):
        tombos.append(lista[tombo])

    for ed in range(2, len(lista), 12):
        ele_des.append(lista[ed])

    for des in range(3, len(lista), 12):
        descricao.append(lista[des])

    for carga in range(4, len(lista), 12):
        cargas.append(lista[carga])

    for c in range(5, len(lista), 12):
        c_contabil.append(lista[c])

    for sala in range(6, len(lista), 12):
        salas.append(lista[sala])

    for data in range(8, len(lista), 12):
        dt_entrada.append(lista[data])
    
    for f in range(10, len(lista), 12):
        fornecedores.append(lista[f])

    for valor in range(11, len(lista), 12):
        valores.append(lista[valor])



    #  montagem do data frame
    df_tabela['tombo'] = tombos
    df_tabela['tombo'] = df_tabela['tombo'].astype(str).str[:-6]

    df_tabela['ed'] = ele_des

    df_tabela['descricao'] = descricao

    df_tabela['carga'] = cargas
    df_tabela['sala'] = salas

    df_tabela['c_contabil'] = c_contabil

    df_tabela['dt_entrada'] = dt_entrada
#     df_tabela['dt_entrada'] = pd.to_datetime(df_tabela['dt_entrada'], infer_datetime_format=True)

    df_tabela['fornecedor'] = fornecedores
    
    df_tabela['valor'] = valores
    df_tabela['valor'] = df_tabela['valor'].str.replace('.','',regex=True)
    df_tabela['valor'] = df_tabela['valor'].str.replace(',','.',regex=True)
    df_tabela['valor'] = pd.to_numeric(df_tabela['valor'])
    df_tabela['valor'] = df_tabela['valor'].round(2)

    return df_tabela

In [58]:
def pagination():
    
    abrir_suap()
    print(f'Extração iniciada as: {datetime.now()}')
    pag = 1
    campi = 12
    saida = False
    complete_df = pd.DataFrame()
    
    
    while saida is False:
    # cafi->12x  crec->4x    cbja->7x   cbar->8x   ccar->11x    ccsa->13x    cgar->10x   cabl->22x 
    # cipj->6x   cjbg->14x   coli->20x  cpes->5x   cplt->21x    cpmr->18    cvsa->9x    reipe->3x
        try:
            nav.get(f'https://suap.ifpe.edu.br/patrimonio/inventario_busca/?page={pag}&status=&empenho=&empenho=&data_final=&sala=&numero_serie=&numero=&rotulos=&data_inicial=&estado_conservacao=6&elemento_despesa=&valor=&responsavel=&responsavel=&contabil=&fornecedor=&fornecedor=&setor_responsavel=&campus={campi}&descricao=')
            celula = nav.find_element(By.CLASS_NAME, 'row1')

            if celula:
                suap_list = table_to_list()
                suap_df = build_df(suap_list)
                complete_df = pd.concat([complete_df, suap_df], ignore_index= True)
                pag += 1

            else:
                saida = True

        except NoSuchElementException:
            
            try:
                error = nav.find_element(By.ID, 'main-frame-error')
                print(f'Caiu as: {datetime.now()} - na pagina {pag}')

                if error:

                    nav.quit()
                    abrir_suap()
                    print(f'Retomou as: {datetime.now()} na pagina {pag}')
                    nav.get(f'https://suap.ifpe.edu.br/patrimonio/inventario_busca/?page={pag}&status=&empenho=&empenho=&data_final=&sala=&numero_serie=&numero=&rotulos=&data_inicial=&estado_conservacao=6&elemento_despesa=&valor=&responsavel=&responsavel=&contabil=&fornecedor=&fornecedor=&setor_responsavel=&campus={campi}&descricao=')
                    celula = nav.find_element(By.CLASS_NAME, 'row1')

                    if celula:
                        pag += 1
                        suap_list = table_to_list()
                        suap_df = build_df(suap_list)
                        complete_df = pd.concat([complete_df, suap_df], ignore_index= True)
                        print(f'pagina pos erro - {pag}')
                        continue

                    else:
                        break
                        
            except NoSuchElementException:
#                 print('acabaram as paginas')
                break
            
                

    print(f'Extração finalizada as: {datetime.now()} na pagina {pag}')
    print(complete_df.info())
    display(complete_df)
    complete_df.to_excel(f'extracoes_suap/{data_e_hora}_{nome}_relatorio_patrimonio.xlsx', index=False)
    
#     nav.quit()
    

In [59]:
def complement_item(item_list):
    
    abrir_suap()
    tombo = []
    status = []
    conservation = []
    carga = []
    desc = []
    empenho = []
    ug_ne = []
    uasg = []
    nf = []
    dt_nf = []
    type_entry = []
    supplier = []
    process = []
    linha = 0
    
    print(f'Iniciou as: {datetime.now()}')
    
#     for linha in item_list:
    while linha is not len(item_list):
        
#         print(linha)
        
        try:        
            nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{item_list[linha]}/")
            
            tombo.append(item_list[linha])
            
            
            st = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[2]/td[2]").get_attribute('innerText')
            status.append(st)

            con = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[4]/td[2]").get_attribute('innerText')
            conservation.append(con)
            
            cargo = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[7]/td[2]").get_attribute('innerText')
            carga.append(cargo)
            # print(carga)

            descricao = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[6]/td[2]/span[1]").get_attribute('innerText')
            desc.append(descricao)

            nav.find_element(By.LINK_TEXT, "Detalhes da Entrada").click()

            try:
                ne = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[12]/td/a").get_attribute('innerText')
                ug = ne[18:22]
                ne = ne[:12]
                empenho.append(ne)
                ug_ne.append(ug)
            # print(empenho)
            except NoSuchElementException as ex:
                empenho.append(indisponivel)
                ug_ne.append(indisponivel)

            try:
                gest = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[1]/td").get_attribute('innerText')
                uasg.append(gest)
            except NoSuchElementException as ex:
                uasg.append(indisponivel)
                # print(uasg)

            try:
                note = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[4]/td").get_attribute('innerText')
                nf.append(note)
            except NoSuchElementException as ex:
                nf.append(indisponivel)
            # print(nf)

            try:
                data = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[5]/td").get_attribute('innerText')
                dt_nf.append(data)
            except NoSuchElementException as ex:
                dt_nf.append(indisponivel)
            # print(data)

            try:
                entry = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[3]/td").get_attribute('innerText')
                type_entry.append(entry)
            except NoSuchElementException as ex:
                type_entry.append(indisponivel)

            try:
                sp = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[6]/td/a").get_attribute('innerText')
                supplier.append(sp)
            except NoSuchElementException as ex:
                supplier.append(indisponivel)

            try:
                pc = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[13]/td").get_attribute('innerText')
                process.append(pc)
            except NoSuchElementException as ex:
                process.append(indisponivel)

        except TimeoutException:
            # print(indisponivel)
            empenho.append(indisponivel)
            ug_ne.append(indisponivel)
            uasg.append(indisponivel)
            nf.append(indisponivel)
            dt_nf.append(indisponivel)
            type_entry.append(indisponivel)
            supplier.append(indisponivel)
            process.append(indisponivel)
            print(f'TimeoutException as: {datetime.now()} no item {item_list[linha]}')

            linha += 1
            continue
        
        except NoSuchElementException as ex:
            
            try:
                error = nav.find_element(By.ID, 'id_username')
                print(f'Caiu as: {datetime.now()} - no item {item_list[linha]}')

                if error:

                    nav.quit()
                    abrir_suap()
                    nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{item_list[linha]}/")
                    print(f'Retomou as: {datetime.now()} no item {item_list[linha]}')
                    continue
            
            except NoSuchElementException as ex:
                print('acabaram as paginas')
                break
            
        linha += 1
        continue

            
    table = pd.DataFrame()
    table['tombo'] = tombo
    table['status'] = status
    table['consetvation'] = conservation
    table['carga'] = carga
    table['desc'] = desc
    
    table['empenho'] = empenho
    table['ug_empenho'] = ug_ne
    table['uasg'] = uasg
    table['nt_fiscal'] = nf
    table['dt_nf'] = dt_nf
    table['tipo_entrada'] = type_entry
    table['fornecedor'] = supplier
    table['processo'] = process
    print(f'Terminou as: {datetime.now()}')
    print(table.info())
    
    table.to_excel(f'{data_e_hora}_{nome}.xlsx', index=False)
        

#     nav.quit()

In [60]:
desf = pd.read_excel('suap/2023-05-25_lista_carga_levantamento_deposito_sem_cadeiras_escolares.xlsx')
desf.head()
complement_item(desf['tombo'])

Iniciou as: 2023-05-26 09:55:43.859039
TimeoutException as: 2023-05-26 09:56:47.607745 no item 19317
TimeoutException as: 2023-05-26 09:57:50.768305 no item 19328
TimeoutException as: 2023-05-26 09:58:53.759795 no item 19329
TimeoutException as: 2023-05-26 09:59:58.451320 no item 19320
TimeoutException as: 2023-05-26 10:01:01.290567 no item 19318
TimeoutException as: 2023-05-26 10:02:05.952641 no item 19327
TimeoutException as: 2023-05-26 10:03:08.701921 no item 19321
TimeoutException as: 2023-05-26 10:04:12.637518 no item 19330
TimeoutException as: 2023-05-26 10:05:16.624555 no item 19319
TimeoutException as: 2023-05-26 10:06:20.739836 no item 19331
TimeoutException as: 2023-05-26 10:07:24.830709 no item 19326
TimeoutException as: 2023-05-26 10:08:29.737819 no item 19322
TimeoutException as: 2023-05-26 10:11:06.854930 no item 19307
TimeoutException as: 2023-05-26 10:14:59.008665 no item 88377
Terminou as: 2023-05-26 10:15:13.132901
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 